In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier


from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import train_test_split
import seaborn as sns


import re
import nltk
from tqdm import tqdm
from nltk.tokenize import WordPunctTokenizer, sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [2]:
negative = pd.read_csv('negative.csv', on_bad_lines='skip', sep=';', header=None) 
positive = pd.read_csv('positive.csv', on_bad_lines='skip', sep=';', header=None)

In [3]:
dataset = pd.concat([negative, positive])
dataset = dataset[[3, 4]]

In [4]:
dataset.columns = ['text', 'label']

In [5]:
def clean_text(text):
    text = text.lower()
    regular = r'[-!$%^&*()_+|~=`{}\[\]:".;<>?\/№«»\r\n\xa0]'
    regular_url = r'(http\S+)|(www\S+)|([\w\d]+www\S+)|([\w\d]+http\S+)'
    text = re.sub(regular, '', text)
    text = re.sub(regular_url, r'URL', text)
    text = re.sub(r'(\d+\s\d+)|(\d+)','', text)
    text = re.sub(r'\s+', ' ', text)
    return text
 

processed_text = []
stop_words = stopwords.words('russian')
lemmatizer = WordNetLemmatizer()
 
for text in dataset['text']: 
    text = clean_text(text)   
    text = word_tokenize(text)       
    text = [word for word in text if word not in stop_words]     
    text = [lemmatizer.lemmatize(w) for w in text]
    processed_text.append(text)


dataset['clean_text'] = processed_text

In [6]:
texts = dataset['text'].values

In [7]:
def normalize(text):
    # print(text)
    ret = [i for i in re.split('\W', text.lower()) if i.isalpha()]
    ret = str(' '.join(ret))
    return ret

def tokenize(text):
    tokenizer = WordPunctTokenizer()
    return tokenizer.tokenize(text)

def lemmatize(tokens):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in tokens]

def remove_stop_word(tokens):
    stopWords = set(stopwords.words('russian'))
    return [i for i in tokens if i not in stopWords]

def preprocess(text):
    ret = normalize(text)
    ret = tokenize(ret)
    ret = lemmatize(ret)
    ret = remove_stop_word(ret)
    return ret

In [8]:
dataset['preprocess_text']= [preprocess(item) for item in tqdm(texts)]

100%|█████████████████████████████████| 226834/226834 [00:25<00:00, 8850.01it/s]


In [9]:
dataset

,text,label,clean_text,preprocess_text
0,на работе был полный пиддес :| и так каждое за...,-1,"[работе, полный, пиддес, каждое, закрытие, мес...","[работе, полный, пиддес, каждое, закрытие, мес..."
1,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,"[коллеги, сидят, рубятся, urban, terror, ,, из...","[коллеги, сидят, рубятся, urban, terror, долба..."
2,@elina_4post как говорят обещаного три года жд...,-1,"[@, elinapost, говорят, обещаного, года, ждут]","[говорят, обещаного, года, ждут]"
3,"Желаю хорошего полёта и удачной посадки,я буду...",-1,"[желаю, хорошего, полёта, удачной, посадки, ,,...","[желаю, хорошего, полёта, удачной, посадки, бу..."
4,"Обновил за каким-то лешим surf, теперь не рабо...",-1,"[обновил, какимто, лешим, surf, ,, работает, п...","[обновил, каким, лешим, surf, работает, просто..."
...,...,...,...,...
114906,"Спала в родительском доме, на своей кровати......",1,"[спала, родительском, доме, ,, своей, кровати,...","[спала, родительском, доме, своей, кровати, пр..."
114907,RT @jebesilofyt: Эх... Мы немного решили сокра...,1,"[rt, @, jebesilofyt, эх, немного, решили, сокр...","[rt, jebesilofyt, эх, немного, решили, сократи..."
114908,"Что происходит со мной, когда в эфире #proacti...",1,"[происходит, мной, ,, эфире, #, proactivefm, з...","[происходит, мной, эфире, proactivefm, звучит,..."
114909,"""Любимая,я подарю тебе эту звезду..."" Имя како...",1,"[любимая, ,, подарю, тебе, звезду, имя, звезды...","[любимая, подарю, тебе, звезду, имя, звезды, п..."
